In [1]:
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier

from keras.utils import plot_model

# from sklearn.model_selection import StratifiedKFold


Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
# Seed the random number generator:
np.random.seed(1)

def load_data(filename, skiprows = 1):
    """
    Function loads data stored in the file filename and returns it as a numpy ndarray.
    
    Inputs:
        filename: given as a string.
        
    Outputs:
        Data contained in the file, returned as a numpy ndarray
    """
    return np.loadtxt(filename, skiprows=skiprows, delimiter=' ')

In [3]:
# Keras NN with sklearn api
def create_model():
    ## Create your own model here given the constraints in the problem
    model = Sequential() # Use np.reshape instead of this in hw
    model.add(Dense(1000, input_dim = 1000))
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    model.add(Dense(500))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense(200))
    model.add(Activation('relu'))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',optimizer='Adam', metrics=['accuracy'])
    
    return model

In [5]:
# Visualiztion
vicotias_secret_model = create_model()
plot_model(vicotias_secret_model, show_shapes=True, to_file='pussy_net_visualization.png')


In [4]:
X = load_data('training_data.txt')
y = X[:, 0]
X = X[:, 1:]

In [5]:
# Short version for testing/
# TODO delete

X_short = X[:100]
y_short = y[:100]


In [6]:
print(np.shape
      (y_short))

print(np.shape(X_short))

(100,)
(100, 1000)


In [7]:
clf1 = LogisticRegression(random_state=1)
clf2 = ExtraTreesClassifier(random_state=1, criterion ='entropy', 
                              max_depth = None, max_features = 10, 
                              min_samples_leaf = 2, min_samples_split = 10, n_estimators = 400)
clf3 = SVC(kernel = 'rbf', C=10, gamma=0.001, probability=True)
clf4 = KerasClassifier(build_fn=create_model, batch_size=128, epochs=2, verbose=0)

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('svc', clf3), ('nn', clf4)], voting='soft')
# eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('svc', clf3)], voting='soft')

In [8]:
# VERY SLOWWWWWW

for clf, label in zip([clf1, clf2, clf3, clf4, eclf], ['Logistic Regression', 'Extra Forest', 'SVC', 'NerualNat', 'Ensemble']):
    scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.84 (+/- 0.00) [Logistic Regression]
Accuracy: 0.85 (+/- 0.01) [Extra Forest]
Accuracy: 0.85 (+/- 0.01) [SVC]
Accuracy: 0.85 (+/- 0.00) [NerualNat]


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/

Accuracy: 0.85 (+/- 0.01) [Ensemble]


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [9]:
eclf.fit(X,y)

VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('rf', ExtraTrees...0.001, verbose=False)), ('nn', <keras.wrappers.scikit_learn.KerasClassifier object at 0x11f8adcc0>)],
         flatten_transform=None, n_jobs=1, voting='soft', weights=None)

In [10]:
X_test = load_data('test_data.txt')

In [11]:
pred = eclf.predict(X_test)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [12]:
print(pred)

[1. 1. 0. ... 0. 1. 0.]


In [13]:
def format_pred(pred):
    result = [int(i) for i in pred]
    with open("4mod_vote_result.txt", "w") as f:
        f.write("Id,Prediction\n") 
        for i in range(1, len(result) + 1):
            f.write(str(i) + "," + str(result[i-1]) + "\n")

In [14]:
format_pred(pred)